# Airbags e outras influências em acidentes fatais 
   ### por Arthur Um, Lucas Lai, e Fernando Chiu


Fonte: National Highway Traffic Safety Administration - uma agência do poder executivo do governo dos EUA, parte do Departamento de Transporte. 
   - NASSCDS - National Automotive Sampling System e o seu Crashworthiness Data System
     - casos selecionados a partir de relatórios de acidentes(batidas) de carro da polícia 
     - melhorias no design de veículos

 <p>  Dados de 1997-2002 sobre acidentes de carro nos EUA em quais ocorreram algum evento prejudicial às vítimas ou suas propriedades, e em que pelo menos um dos veículos envolvidos teve que ser tirado a guincho. Os dados estão limitados a apenas os passageiros que sentaram nos bancos dianteiros do carro, entre outras limitações.</p>


In [3]:
df<-read.csv("Car accidents.csv")
str(df)

'data.frame':	26217 obs. of  16 variables:
 $ X          : int  1 2 3 4 5 6 7 8 9 10 ...
 $ dvcat      : Factor w/ 5 levels "1-9km/h","10-24",..: 3 2 2 3 3 4 5 5 2 2 ...
 $ weight     : num  25.1 25.1 32.4 495.4 25.1 ...
 $ dead       : Factor w/ 2 levels "alive","dead": 1 1 1 1 1 1 1 2 1 1 ...
 $ airbag     : Factor w/ 2 levels "airbag","none": 2 1 2 1 2 2 2 2 2 2 ...
 $ seatbelt   : Factor w/ 2 levels "belted","none": 1 1 2 1 1 1 1 2 1 1 ...
 $ frontal    : int  1 1 1 1 1 1 1 1 0 1 ...
 $ sex        : Factor w/ 2 levels "f","m": 1 1 1 1 1 1 2 2 2 1 ...
 $ ageOFocc   : int  26 72 69 53 32 22 22 32 40 18 ...
 $ yearacc    : int  1997 1997 1997 1997 1997 1997 1997 1997 1997 1997 ...
 $ yearVeh    : int  1990 1995 1988 1995 1988 1985 1984 1987 1984 1987 ...
 $ abcat      : Factor w/ 3 levels "deploy","nodeploy",..: 3 1 3 1 3 3 3 3 3 3 ...
 $ occRole    : Factor w/ 2 levels "driver","pass": 1 1 1 1 1 1 1 1 1 1 ...
 $ deploy     : int  0 1 0 1 0 0 0 0 0 0 ...
 $ injSeverity: int  3 1 4 1 3

- dvcat - velocidade do impacto estimado (1-9km/h, 10-24, 25-39, 40-54, 55+).
- weight - o peso das observações, porém não tão precisos.*
- dead - se a vítima observada faleceu ou não (alive dead).
- airbag - se o veículo observado possuía airbag ou não (none airbag).
- *seatbelt* - se a vítima observada estava com cinto ou não (none belted).
- frontal - se o impacto da observação foi frontal ou não (numérico;  0 = não frontal, 1=frontal).
- sex - sexo da vítima observada (f m).
- ageOFocc - idade do ocupante da observação em anos.
- yearacc - ano em que o acidente ocorreu.
- yearVeh - ano do modelo do carro.
- abcat - um ou mais airbags foram ativados?(motorista ou passageiro; deploy nodeploy unavail)
- occRole - papel do ocupante da observação (driver pass).
- deploy - mesmo que abcat, porém numérico (0 caso não houve airbag, 1 caso houve).
- injSeverity - severidade do ferimento da vítima observada, numérico; 0:nenhum, 1:possivelmente ferido, 2:incapacitação, 3:incapacitado, 4:morto; 5: desconhecido, 6:morte precedente
- caseid - tipo: character; populations sampling unit + case number + vehicle number. Podemos utilizá-lo para identificar o veículo como único em cada ano.


In [4]:
summary(df)

       X             dvcat           weight            dead      
 Min.   :    1   1-9km/h:  686   Min.   :    0.00   alive:25037  
 1st Qu.: 6555   10-24  :12848   1st Qu.:   32.47   dead : 1180  
 Median :13109   25-39  : 8214   Median :   86.99                
 Mean   :13109   40-54  : 2977   Mean   :  462.81                
 3rd Qu.:19663   55+    : 1492   3rd Qu.:  364.72                
 Max.   :26217                   Max.   :57871.59                
                                                                 
    airbag        seatbelt        frontal       sex          ageOFocc    
 airbag:14419   belted:18573   Min.   :0.0000   f:12248   Min.   :16.00  
 none  :11798   none  : 7644   1st Qu.:0.0000   m:13969   1st Qu.:22.00  
                               Median :1.0000             Median :33.00  
                               Mean   :0.6433             Mean   :37.21  
                               3rd Qu.:1.0000             3rd Qu.:48.00  
                            

In [7]:
colSums(is.na(df))

X       dvcat      weight        dead      airbag    seatbelt 
          0           0           0           0           0           0 
    frontal         sex    ageOFocc     yearacc     yearVeh       abcat 
          0           0           0           0           1           0 
    occRole      deploy injSeverity      caseid 
          0           0         153           0

In [16]:
df<-na.omit(df) # Excluimos os valores na
table(is.na(df))
message("No. de obs.:")
nrow(df) # Antes era 26217 


 FALSE 
417008 

No. de obs.:


[1] 26063